In [1]:
pip install pygame 


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pygame
import copy 
import random 
import random
import math 
import time 
from copy import deepcopy


WIDTH, HEIGHT = 800, 800 # 800 pixels high by 800 pixels wide
ROWS, COLS = 8, 8 # 8 rows by 8 columns = std checkers board
SQUARE_SIZE = WIDTH//COLS # defines the area of one square

# RGB Color Scheme for Pygame
RED = (255, 0, 0) # define red color for pygame
WHITE = (255, 255, 255) # define white
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GRAY = (128, 128, 128)

CROWN = pygame.transform.scale(pygame.image.load('/Users/ssingson/Desktop/crown.png'), (44, 25))

class Piece:
    PADDING = 20
    OUTLINE = 2

    def __init__(self, row, col, color):
        self.row = row
        self.col = col
        self.colorname = color
        if color == 'White':
            self.color = WHITE
        else: 
            self.color = RED                
        self.king = False
        self.x = 0
        self.y = 0
        self.calc_pos()

    def calc_pos(self): # calculate position based on the row and column we are in
        self.x = SQUARE_SIZE * self.col + SQUARE_SIZE // 2
        self.y = SQUARE_SIZE * self.row + SQUARE_SIZE // 2

    def make_king(self):
        self.king = True

    def isKing(self):   # Bugfix to only allow a piece.make_king() to happen only once
        return self.king == True

    def draw(self, win):
        radius = SQUARE_SIZE // 2 - self.PADDING
        pygame.draw.circle(win, GRAY, (self.x, self.y), radius + self.OUTLINE)
        pygame.draw.circle(win, self.color, (self.x, self.y), radius)
        if self.king:
            win.blit(CROWN, (self.x - CROWN.get_width()//2, self.y - CROWN.get_height()//2))

    def move(self, row, col):
        self.row = row
        self.col = col
        self.calc_pos()

    def __repr__(self):
        return str(self.color)
    
class Board:
    def __init__(self):
        self.board = []
        self.red_left = self.white_left = 12 # define the amount of red and black pieces each player begins with
        self.red_kings = self.white_kings = 0 # no one starts with king pieces
        self.create_board()

    def draw_squares(self, win):
        win.fill(BLACK)
        for row in range(ROWS):
            for col in range(row % 2, COLS, 2): # even indices are red, odd are black
                pygame.draw.rect(win, RED, (row * SQUARE_SIZE, col * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
    
    # AI definition #1 for Minimax: White = AI
    def evaluate(self, player): # Evaluate score of the board (AI difficulty)
        if player == 'White': 
            return self.white_left - self.red_left + (self.white_kings * 1.5 - self.red_kings * 1.5) # Edit: Increase weight of King Pieces
        else: 
            return self.red_left - self.white_left + (self.red_kings * 1.5 - self.white_kings * 1.5) # Edit: Increase weight of King Pieces


    # AI Minimax Function #2:
    def get_all_pieces(self, colorname):

        pieces = []
        for row in self.board:
            for piece in row:
                if piece != 0 and piece.colorname == colorname:
                    pieces.append(piece)
        return pieces

    def move(self, piece, row, col):
        self.board[piece.row][piece.col], self.board[row][col] = self.board[row][col], self.board[piece.row][piece.col]
        piece.move(row, col)
        if (row == ROWS - 1 or row == 0):   # last or 1st row of the board, make piece a KING
            aKing = piece.isKing()          # Check If Piece is already a King Piece (affects Minimax Score)
            if aKing == False:
                piece.make_king()
                if piece.color == WHITE:
                    self.white_kings += 1
                else:
                    self.red_kings += 1

    def get_piece(self, row, col):                
        return self.board[row][col]

    def create_board(self): # draw out checker piece arrangements: 3 total rows of 12 total pieces
        for row in range(ROWS):
            self.board.append([])
            for col in range(COLS):
                if col % 2 == ((row + 1) % 2):
                    if row < 3:
                        self.board[row].append(Piece(row, col, 'White'))
                    elif row > 4:
                        self.board[row].append(Piece(row, col, 'Red'))
                    else: # add a zero instead to keep track of the board
                        self.board[row].append(0)
                else:
                    self.board[row].append(0)

    def draw(self, win):
        self.draw_squares(win)
        for row in range(ROWS):
            for col in range(COLS):
                piece = self.board[row][col]
                if piece != 0:
                    piece.draw(win)

    def remove(self, pieces):
        for piece in pieces:
            self.board[piece.row][piece.col] = 0
            if piece != 0:
                if piece.color == RED:
                    self.red_left = self.red_left - 1
                else:
                    self.white_left = self.white_left - 1

    def winner(self):
        if self.red_left <= 0:
            return 'White'
        elif self.white_left <= 0:
            return 'Red'
        else:
            return None

    def get_valid_moves(self, piece):
        moves = {}
        left  = piece.col - 1
        right = piece.col + 1
        row = piece.row

        if piece.color == RED or piece.king: # determine valid direction of movement
            moves.update(self._traverse_left(row - 1, max(row - 3, -1), -1, piece.color, left))
            moves.update(self._traverse_right(row - 1, max(row - 3, -1), -1, piece.color, right))
        
        if piece.color == WHITE or piece.king:
            moves.update(self._traverse_left(row + 1, min(row + 3, ROWS), 1, piece.color, left))
            moves.update(self._traverse_right(row + 1, min(row + 3, ROWS), 1, piece.color, right))

        return moves # returns a dictionary

    def get_valid_moves_all(self, colorname): 
        moves = {}
        for piece in self.get_all_pieces(colorname): 
            left  = piece.col - 1
            right = piece.col + 1
            row = piece.row
            if piece.color == RED or piece.king: # determine valid direction of movement
                moves.update(self._traverse_left(row - 1, max(row - 3, -1), -1, piece.color, left))
                moves.update(self._traverse_right(row - 1, max(row - 3, -1), -1, piece.color, right))

            if piece.color == WHITE or piece.king:
                moves.update(self._traverse_left(row + 1, min(row + 3, ROWS), 1, piece.color, left))
                moves.update(self._traverse_right(row + 1, min(row + 3, ROWS), 1, piece.color, right))
        return moves
            
    def _traverse_left(self, start, stop, step, color, left, skipped=[]): # step = up or down
        moves = {}
        last = []
        for r in range(start, stop, step):
            if left < 0:
                break

            current = self.board[r][left]
            if current == 0: # found an empty square
                if skipped and not last:
                    break
                elif skipped:
                    moves[(r, left)] = last + skipped
                else:
                    moves[(r, left)] = last
                
                if last:
                    if step == -1:
                        row = max(r - 3, 0)
                    else:
                        row = min(r + 3, ROWS)
                    moves.update(self._traverse_left(r + step, row, step, color, left - 1, skipped=last))
                    moves.update(self._traverse_right(r + step, row, step, color, left + 1, skipped=last))
                break # just in case ??
            elif current.color == color: # if its a friendly square
                break # do nothing, not a valid move
            else:
                last = [current]

            left -= 1
        return moves

    def _traverse_right(self, start, stop, step, color, right, skipped=[]):
        moves = {}
        last = []
        for r in range(start, stop, step):
            if right >= COLS:
                break

            current = self.board[r][right]
            if current == 0: # found an empty square
                if skipped and not last:
                    break
                elif skipped:
                    moves[(r, right)] = last + skipped
                else:
                    moves[(r, right)] = last
                
                if last:
                    if step == -1:
                        row = max(r - 3, 0)
                    else:
                        row = min(r + 3, ROWS)
                    moves.update(self._traverse_left(r + step, row, step, color, right - 1, skipped=last))
                    moves.update(self._traverse_right(r + step, row, step, color, right + 1, skipped=last))
                break 
            elif current.color == color: # if its a friendly square
                break # do nothing, not a valid move
            else:
                last = [current]
            
            right += 1
        return moves

# Game Class interfaces with the board
# Keeps game separate from main loop checking buttons, movement, etc
class Game:
    def __init__(self, win, showpygame = False): # could define game window in here, BUT in the case we want more than one instance running at the same time
        self._init()
        self.showpygame = showpygame
        if self.showpygame == True:
            self.win = win # game window

    def update(self):
        if self.showpygame == True: 
            self.board.draw(self.win)
        self.draw_valid_moves(self.valid_moves)
        pygame.display.update()
    
    def _init(self):
        self.selected = None
        self.board = Board()
        self.turn = 'Red'
        self.valid_moves = {} # dictionary to define legal moves

    def winner(self):
        return self.board.winner()

    def reset(self): # user reset to start a new game
        self._init()

    def select(self, row, col): # determies is a move can be performed or not
        if self.selected: # if the piece is already selected
            result = self._move(row, col) # move selected piece to row/col
            if not result: # if move is not legal (wrong space, etc)
                self.selected = None # reset the selection
                self.select(row, col) # reselect a row and column
        
        piece = self.board.get_piece(row, col)
        if piece != 0 and piece.color == self.turn:
            self.selected = piece
            self.valid_moves = self.board.get_valid_moves(piece)
            return True # valid selection returns true

        return False # Invalid selection returns false.

    def _move(self, row, col): # private: used by select only
        piece = self.board.get_piece(row, col)
        if self.selected and piece == 0 and (row, col) in self.valid_moves: # can only move to empty space == 0
            self.board.move(self.selected, row, col)
            skipped = self.valid_moves[(row, col)]
            if skipped:
                self.board.remove(skipped)
            self.change_turn()
        else: 
            return False

        return True # move is successful

    
    def draw_valid_moves(self, moves): # loop through dictionary
        for move in moves:
            row, col = move
            if self.showpygame == True: 
                pygame.draw.circle(self.win, BLUE, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), 15)

    def change_turn(self):
        self.valid_moves = {}
        if self.turn == 'Red':
            self.turn = 'White'
        else:
            self.turn = 'Red'

    # AI Function definition # 3
    def get_board(self):
        return self.board

    # AI Function definition # 4
    def ai_move(self, board):
        self.board = board
        self.change_turn()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# File for Minimax AI Algo
from copy import deepcopy
import pygame

RED = (255, 0, 0)
WHITE = (255, 255, 255)

def simulate_move(piece, move, board, game, skip):
    board.move(piece, move[0], move[1])
    if skip:
        board.remove(skip)

    return board

def get_all_moves(board, color, game):
    moves = []

    for piece in board.get_all_pieces(color):
        valid_moves = board.get_valid_moves(piece)
        for move, skip in valid_moves.items():
            temp_board = deepcopy(board)
            temp_piece = temp_board.get_piece(piece.row, piece.col)
            new_board = simulate_move(temp_piece, move, temp_board, game, skip)
            moves.append(new_board)

    return moves

def get_next_move(old_board, new_board, color): 
    old_board_pieces = []
    new_board_pieces = []
    for piece in old_board.get_all_pieces(color): 
        old_board_pieces.append([piece.row, piece.col])
    for piece in new_board.get_all_pieces(color): 
        new_board_pieces.append([piece.row, piece.col])
    return [item for item in old_board_pieces + new_board_pieces if (old_board_pieces.count(item) == 1) ^ (new_board_pieces.count(item) == 1)]



def minimax(position, depth, color, game):
    if color == 'White':
        player = 'White'
        oppenent = 'Red'
    else: 
        player = 'Red'
        opponent = 'White'
    if depth == 0 or position.winner() != None:
        return position.evaluate(color), position

    if player:
        maxEval = -100000000 
        best_move = None
        for move in get_all_moves(position, player, game):
            evaluation = minimax(move, depth - 1, True, game)[0] 
            maxEval = max(maxEval, evaluation)
            if maxEval == evaluation:
                best_move = move

        return maxEval, best_move

    else:
        minEval = -100000000 
        best_move = None
        for move in get_all_moves(position, opponent, game):
            evaluation = minimax(move, depth - 1, False, game)[0] 
            minEval = max(minEval, evaluation)
            if minEval == evaluation:
                best_move = move

        return minEval, best_move

In [7]:
class MCTSAgentsAll:
    def __init__(self, max_iterations, beta = 0.5, exploration_factor = 1.4, model = 'MCTS'):
        self.max_iterations = max_iterations
        self.beta = beta
        self.model = model
        self.exploration_factor = exploration_factor
        
    def get_action(self, game, player, board):
        root = Node(game = game, board = board, player = player)
        for i in range(self.max_iterations):
            node = root
            temp_board = copy.deepcopy(board)
            path = []

            # Selection
            while node.untried_actions == [] and node.children != []:
                node = self.select_child(node, path, expl_factor = self.exploration_factor)
                temp_board = node.board
                path.append(node)
 
            # Expansion
            if node.untried_actions != []:
                temp_board = random.choice(node.untried_actions)
                node = node.add_child(board = temp_board, game = game, player = player)
            
            # Simulation
            iterations = 0
            temp_player = copy.copy(player)
            
            while temp_board.winner() == None and iterations < 20:               
                if self.model == 'heuristic': 
                    actions = node.untried_actions
                    action = self.select_best_action(node, temp_player)
                    
                else: 
                    available_moves = get_all_moves(temp_board, temp_player, game)
                    if available_moves != []:
                        temp_board = random.choice(available_moves) 
                if temp_player == 'Red': 
                    temp_player = 'White'
                else: 
                    temp_player = 'Red'
                iterations += 1
                path.append(node)
            round_winner = temp_board.winner()
            if round_winner is None: 

                if temp_board.evaluate('Red') > temp_board.evaluate('White'): 
                    round_winner = 'Red'
                else:
                    round_winner = 'White'
            #Rave Scoring
            for path_node in path:
                path_node.rave_visits += 1
                if (round_winner == node.player) and (node.move == path_node.move): 
                    path_node.rave_wins +=1

            # Backpropagation
            while node is not None:
                node.visits = node.visits + 1
                if round_winner == node.player:
                    node.wins += 1 

                node = node.parent
        best_child = self.select_child(root, path, expl_factor = 0)
        return best_child.board
    
    def select_child(self, node, path, expl_factor):
        total_visits = sum(child.visits for child in node.children)
        best_score = -100000000
        best_child = None

        for child in node.children:
            exploitation = child.wins / child.visits
            exploration = math.sqrt(math.log(total_visits) / child.visits)
            if self.model != 'mcts': 
                rave_exploitation = child.rave_wins / child.rave_visits if child.rave_visits > 0 else 0
                try: 
                    rave_exploration = math.sqrt(math.log(sum(n.rave_visits for n in path)) / child.rave_visits) if child.rave_visits > 0 else 0
                except: 
                    rave_exploration = 0
                score = (1 - self.beta) * exploitation + self.beta * rave_exploitation + expl_factor * (1 - self.beta) * exploration + expl_factor * self.beta * rave_exploration
            else: 
                score = exploitation + expl_factor * exploration
            if score > best_score:
                best_score = score
                best_child = child
        return best_child
    
    def add_child(self, state, player, action = None, move = None):
        child_node = node(state, game, state, action, player, move)
        self.children.append(child_node)
        return child_node
    
    def select_best_action(self, node, player):
        # Choose the action that best satisfies the heuristics
        best_action = None
        best_score = -100000000

        for action in node.untried_actions:
            score = self.evaluate_heuristics(node, action, player)
            if score > best_score:
                best_score = score
                best_action = action
        return best_action

    def evaluate_heuristics(self, node, action, player):
        # Heuristics:
        # 1. Capture pieces
        # 2. Do not lose pieces 
        # 3. Crown your pieces to kings
        # 4. Move your home row checkers only when you need them 

        score = 0

        # Heuristic 1: Capture Piece
        if ((node.player == 'White') and (action.red_left < node.board.red_left)) or ((node.player == 'Red') and (action.white_left < node.board.white_left)):
            score += 1
            
        # Heuristic 2: Lost Piece
        if ((node.player == 'Red') and (action.white_left < node.board.white_left)) or ((node.player == 'White') and (action.red_left < node.board.red_left)):
            score -= 1

        # Heuristic 3: Crown your pieces to kings
        if ((node.player == 'White') and (action.red_kings > node.board.red_kings)) or ((node.player == 'Red') and (action.white_kings > node.board.white_kings)):
            score += 1
            
        if ((node.player == 'Red') and (action.white_kings > node.board.white_kings)) or ((node.player == 'White') and (action.red_kings > node.board.red_kings)):
            score -= 1

        # Heuristic 4: Move your home row checkers only when you need them
        if player == 'White':
            home_row = 7
        else: 
            home_row = 0 
        for piece in action.get_all_pieces(node.player): 
            if piece.row == home_row:
                score += 1
        for piece in node.board.get_all_pieces(node.player): 
            if piece.row == home_row:
                score += 1

        return score

class Node:
    def __init__(self, game, board, player, parent=None, action=None, move = None):
        self.board = board
        self.parent = parent
        self.action = action
        self.move = move
        self.children = []
        self.wins = 0
        self.rave_wins = 0
        self.rave_visits = 0
        self.visits = 0
        self.player = game.turn
        self.untried_actions = get_all_moves(board, player, game)

    def add_child(self, board, game, player, move = None):
        node = Node(board = board, game = game, player = player, parent=self, move = get_next_move(self.board, board, player))
        self.children.append(node)
        self.untried_actions.remove(board)
        return node

In [8]:
# Create a main loop that checks for user input (mouse, keyboard, etc) 
# Draw all the pieces, the board, etc.



def get_row_col_from_mouse(pos):
    x, y = pos
    row = y // SQUARE_SIZE
    col = x // SQUARE_SIZE
    return row, col


def main(showpygame = False, white_player = 'human', white_beta = 0.5, white_exploration_factor = 1.4, white_minimax_moves = 3, red_player = 'human', red_beta = 0.5, red_exploration_factor = 1.4, red_minimax_moves = 3, white_mcts_iterations = 100, red_mcts_iterations = 100): 
    white_time = 0
    red_time = 0

    FPS = 60 # do we need this in the constants file? No, only references drawing the game
    WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
    if showpygame == True: 
        
        # set caption for display here: shows up in top title bar
        pygame.display.set_caption('AI Minimax Checkers')

    moves = 0 
    
    run = True
    game = Game(WINDOW, showpygame = showpygame)
    
    game_winner = None
    while run:
        try: 
            if game.turn == 'White': 
                start_time = time.time()
                if white_player == 'human':
                    pass
                elif white_player == 'minimax': 
                    gameeval, new_board = minimax(game.get_board(), white_minimax_moves, 'White', game)
                    game.ai_move(new_board)
                else: 
                    bestmove = MCTSAgentsAll(white_mcts_iterations, model = white_player, beta = white_beta, exploration_factor = white_exploration_factor)
                    new_board = bestmove.get_action(game = game, player = 'White', board = game.get_board())
                    game.ai_move(new_board)
                white_time = white_time + time.time() - start_time
            elif game.turn == 'Red': 
                start_time = time.time()
                if red_player == 'human':
                    pass
                elif red_player == 'minimax': 
                    gameeval, new_board = minimax(game.get_board(), red_minimax_moves, 'Red', game)
                    game.ai_move(new_board)
                else: 
                    bestmove = MCTSAgentsAll(red_mcts_iterations, model = red_player, beta = red_beta, exploration_factor = red_exploration_factor)
                    new_board = bestmove.get_action(game = game, player = 'Red', board = game.get_board())
                    game.ai_move(new_board)
                red_time = red_time + time.time() - start_time
            moves += 1
        except Exception as e: 
            print(e)
            game_winner = 'Draw'
            print('Winner:', game_winner)
            return {'Winner': game_winner, 'White Player': white_player, 'Red Player': red_player, 'Total Moves': moves, 'Red Pieces Left': new_board.red_left, 'White Pieces Left': new_board.white_left, 'Red Kings': new_board.red_kings, 'White Kings': new_board.white_kings, 'Red Beta': red_beta, 'White Beta': white_beta, 'Red Exploration Factor': red_exploration_factor, 'White Exploration Factor': white_exploration_factor, 'White Time': white_time, 'Red Time': red_time, 'White Minimax Moves': white_minimax_moves, 'Red Minimax Moves': red_minimax_moves, 'White MCTS Iterations': white_mcts_iterations, 'Red MCTS Iterations': red_mcts_iterations} 
        if moves == 300: 
            game_winner = 'Draw'
            print('Winner:', game_winner)
            return {'Winner': game_winner, 'White Player': white_player, 'Red Player': red_player, 'Total Moves': moves, 'Red Pieces Left': new_board.red_left, 'White Pieces Left': new_board.white_left, 'Red Kings': new_board.red_kings, 'White Kings': new_board.white_kings, 'Red Beta': red_beta, 'White Beta': white_beta, 'Red Exploration Factor': red_exploration_factor, 'White Exploration Factor': white_exploration_factor, 'White Time': white_time, 'Red Time': red_time, 'White Minimax Moves': white_minimax_moves, 'Red Minimax Moves': red_minimax_moves, 'White MCTS Iterations': white_mcts_iterations, 'Red MCTS Iterations': red_mcts_iterations} 

        if game_winner == 'Draw': 
             
            run = False
            
        if game.winner() != None:
            game_winner = game.winner()
            print('Winner:', game_winner)
            run = False
            
        if showpygame == True: 
          for event in pygame.event.get():
              if event.type == pygame.QUIT:
                  run = False 

              if event.type == pygame.MOUSEBUTTONDOWN: # Select game piece with mouse
                  pos = pygame.mouse.get_pos()
                  row, col = get_row_col_from_mouse(pos)
                  game.select(row, col)

        game.update()

    pygame.quit()
    return {'Winner': game_winner, 'White Player': white_player, 'Red Player': red_player, 'Total Moves': moves, 'Red Pieces Left': new_board.red_left, 'White Pieces Left': new_board.white_left, 'Red Kings': new_board.red_kings, 'White Kings': new_board.white_kings, 'Red Beta': red_beta, 'White Beta': white_beta, 'Red Exploration Factor': red_exploration_factor, 'White Exploration Factor': white_exploration_factor, 'White Time': white_time, 'Red Time': red_time, 'White Minimax Moves': white_minimax_moves, 'Red Minimax Moves': red_minimax_moves, 'White MCTS Iterations': white_mcts_iterations, 'Red MCTS Iterations': red_mcts_iterations} 



#main(showpygame = True, white_player = 'minimax', red_player = 'mcts')

In [ ]:
columns = ['Winner', 'White Player', 'Red Player', 'Total Moves', 'Red Pieces Left', 'White Pieces Left', 'Red Kings', 'White Kings', 'Red Beta', 'White Beta', 'Red Exploration Factor', 'White Exploration Factor', 'White Time', 'Red Time', 'White Minimax Moves', 'Red Minimax Moves'] 
players = ['minimax', 'mcts', 'rave', 'heuristic']
iterations = [10,50,100,500,1000]

# If new file, create new dataframe, otherwise comment out the new Dataframe line to pull the csv being used 
checkers_games = pd.DataFrame(columns = columns)
#checkers_games = pd.read_csv('/Users/ssingson/Desktop/Checkers Games3.csv')

for i in range(5000): 
    print('Game:', i)
    checkers_games = checkers_games.append(main(showpygame = False, \
                        white_player = random.choice(players), \
                        white_beta = random.uniform(0, 1), \
                        white_exploration_factor = random.uniform(0, 5), \
                        white_minimax_moves = random.randint(2,4),\
                        white_mcts_iterations = random.choice(iterations), \
                        red_player = random.choice(players), \
                        red_beta = random.uniform(0,1), \
                        red_exploration_factor = random.uniform(0,5), \
                        red_minimax_moves = random.randint(2,4), \
                        red_mcts_iterations = random.choice(iterations)), ignore_index=True)
    
    checkers_games.to_csv('/Users/ssingson/Desktop/Checkers Games3.csv')

Game: 0
'NoneType' object has no attribute 'board'
Winner: Draw
Game: 1


/var/folders/gl/rc7yhs2902ngym_48591zj980000gp/T/ipykernel_60468/1177224369.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  checkers_games = checkers_games.append(main(showpygame = False, \


Winner: White
Game: 2


/var/folders/gl/rc7yhs2902ngym_48591zj980000gp/T/ipykernel_60468/1177224369.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  checkers_games = checkers_games.append(main(showpygame = False, \


In [ ]:
new_board.get_all_pieces('White')